<a href="https://colab.research.google.com/github/piyush91295/TCS_iON_Internship/blob/main/TCS_INTERNSHIP_PIYUSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#                    TCS iON: Rio 45- Automate detection of different sentiments from                                                       textual comments and feedback


#                    PIYUSH TRIVEDI
#                    IIT KANPUR

# Importing Modules

In [32]:
import tensorflow as tf
import numpy as np
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Checking the Tensorflow Version

In [33]:
tf.__version__

'2.13.0'

## Reading in the Twitter Data obtained from Kaggle

### Data Link: Charan Gowda, Anirudh, Akshay Pai, &amp; Chaithanya kumar A. (2019). <i>Twitter and Reddit Sentimental analysis Dataset</i> [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DS/429085

## Here I am performing sentiment analysis only on the twitter data.

In [34]:
df = pd.read_csv('Twitter_Data.csv')

### Checking the Head of the Data

In [35]:
df.head(10)

,Clean_Text,Category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
5,kiya tho refresh maarkefir comment karo,0.0
6,surat women perform yagna seeks divine grace f...,0.0
7,this comes from cabinet which has scholars lik...,0.0
8,with upcoming election india saga going import...,1.0
9,gandhi was gay does modi,1.0


## Code : Sentiment
##  1  : Positive ¶
##  0  : Neutral
##  -1 : Negative

In [36]:
df.shape

(162980, 2)

### Reviewing the Columns in the Data

In [37]:
df.columns

Index(['Clean_Text', 'Category'], dtype='object')

In [38]:
df.index

RangeIndex(start=0, stop=162980, step=1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Clean_Text  162976 non-null  object 
 1   Category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


### Checking the NaN values in the Clean_Text Column

In [40]:
df[df['Clean_Text'].isnull()]

,Clean_Text,Category
148,NaN,0.0
158694,NaN,-1.0
159443,NaN,0.0
160560,NaN,1.0


### Checking the NaN values in the Category Column

In [41]:
df[df['Category'].isnull()]

,Clean_Text,Category
130448,the foundation stone northeast gas grid inaugu...,NaN
155642,dear terrorists you can run but you cant hide ...,NaN
155698,offense the best defence with mission shakti m...,NaN
155770,have always heard politicians backing out thei...,NaN
158693,modi government plans felicitate the faceless ...,NaN
159442,chidambaram gives praises modinomics,NaN
160559,the reason why modi contested from seats 2014 ...,NaN


In [42]:
df.isnull().sum()

Clean_Text    4
Category      7
dtype: int64

### Dropping the NaN Values

In [43]:
df = df.dropna()

In [44]:
df

,Clean_Text,Category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [45]:
df['Clean_Text'] = df['Clean_Text'].astype('str')

<ipython-input-45-2abd2cee8cbc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Clean_Text'] = df['Clean_Text'].astype('str')


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162969 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Clean_Text  162969 non-null  object 
 1   Category    162969 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.7+ MB


### Dataframe information after dropping the NaN values

In [47]:
df.isnull().any()

Clean_Text    False
Category      False
dtype: bool

In [48]:
X = df['Clean_Text']
y = df['Category']

In [49]:
# Changing the Datatype to float64
y = y.apply(pd.to_numeric)

In [50]:
X

0         when modi promised “minimum government maximum...
1         talk all the nonsense and continue all the dra...
2         what did just say vote for modi  welcome bjp t...
3         asking his supporters prefix chowkidar their n...
4         answer who among these the most powerful world...
                                ...                        
162975    why these 456 crores paid neerav modi not reco...
162976    dear rss terrorist payal gawar what about modi...
162977    did you cover her interaction forum where she ...
162978    there big project came into india modi dream p...
162979    have you ever listen about like gurukul where ...
Name: Clean_Text, Length: 162969, dtype: object

In [51]:
y

0        -1.0
1         0.0
2         1.0
3         1.0
4         1.0
         ... 
162975   -1.0
162976   -1.0
162977    0.0
162978    0.0
162979    1.0
Name: Category, Length: 162969, dtype: float64

## Importing nltk and stopwords

In [52]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


## Importing modules

In [53]:
import random
import string
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import twitter_samples
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Text Cleaning and Preprocessing

## Removing stopwords and punctuation

In [54]:
#Importing the english stop words list from NLTK
stopwords_english = stopwords.words('english')

print('Stop words\n')
print(stopwords_english)

print('\nPunctuation\n')
print(string.punctuation)


Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [55]:
df=df.dropna(axis=0)

In [56]:
df.isnull().sum()

Clean_Text    0
Category      0
dtype: int64

In [57]:
# Removing the stopwords from clean texts
df['Clean_Text'] = df['Clean_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_english)]))

In [58]:
df['Clean_Text'][0]

'modi promised “minimum government maximum governance” expected begin difficult job reforming state take years get justice state business exit psus temples'

In [59]:
# Removing the punctuations from tweets
df['Clean_Text'] = df['Clean_Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [60]:
df['Clean_Text'][0]

'modi promised minimum government maximum governance expected begin difficult job reforming state take years get justice state business exit psus temples'

## Stemming the words using Porter Stemmer

In [61]:
PS = PorterStemmer()
def stemming(word):
    Corpus=[]
    for i in word.split():
        Corpus.append(PS.stem(i))
    return ' '.join(Corpus)

df['Clean_Text'] = df['Clean_Text'].apply(lambda x:stemming(x))

In [62]:
df['Category'] = [2 if x == -1 else x for x in df['Category']]

In [63]:
df_1=df.copy()

In [64]:
df_1

,Clean_Text,Category
0,modi promis minimum govern maximum govern expe...,2.0
1,talk nonsens continu drama vote modi,0.0
2,say vote modi welcom bjp told rahul main campa...,1.0
3,ask support prefix chowkidar name modi great s...,1.0
4,answer among power world leader today trump pu...,1.0
...,...,...
162975,456 crore paid neerav modi recov congress lead...,2.0
162976,dear rss terrorist payal gawar modi kill 1000 ...,2.0
162977,cover interact forum left,0.0
162978,big project came india modi dream project happ...,0.0


## Importing the necessary modules

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

## Tokenizing the words using Tokenizer

In [66]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Clean_Text)
word_index = tokenizer.word_index

In [67]:
vocab_size = len(word_index)+1

## Padding the Tweets

In [68]:
# Padding the tokenized sequences to the same length

max_length = 200
lines = pad_sequences(tokenizer.texts_to_sequences(df.Clean_Text),
                        maxlen = max_length)

In [69]:
df.Clean_Text = lines.tolist()

In [70]:
df.Category.value_counts()

1.0    72249
0.0    55211
2.0    35509
Name: Category, dtype: int64

In [71]:
df.head(10)

,Clean_Text,Category
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0
9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0


In [72]:
df.isnull().sum()

Clean_Text    0
Category      0
dtype: int64

## Splitting the Data

In [73]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['Clean_Text'],df['Category'],test_size=0.2,random_state=101)
X_train = np.vstack(X_train.values)
y_train = np.vstack(y_train.values)
X_val = np.vstack(X_test.values)
y_val = np.vstack(y_test.values)


## Doing the Sequential Modelling

In [74]:
model = tf.keras.Sequential()

# Adding the Input layer
model.add(Input(shape=(None,)))

# Adding the Embedding layer
model.add(Embedding(input_dim=vocab_size,output_dim=200,trainable=True))

# Adding the LSTM layer
model.add(LSTM(64, activation='relu'))

# Adding the Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Adding the Output layer
model.add(Dense(3, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 200)         17711000  
                                                                 
 lstm (LSTM)                 (None, 64)                67840     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 17783195 (67.84 MB)
Trainable params: 17783195 (67.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Adding the optimizer and the loss function

## Training the model on train data

In [75]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, batch_size=200, verbose=1, shuffle=True, validation_data=(X_val,y_val))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


652/652 [==============================] - 302s 452ms/step - loss: 30452836.0000 - accuracy: 0.6591 - val_loss: 0.5599 - val_accuracy: 0.7791
Epoch 2/5
652/652 [==============================] - 252s 386ms/step - loss: 0.4266 - accuracy: 0.8442 - val_loss: 0.4714 - val_accuracy: 0.8237
Epoch 3/5
652/652 [==============================] - 244s 373ms/step - loss: 0.2970 - accuracy: 0.8971 - val_loss: 0.4908 - val_accuracy: 0.8267
Epoch 4/5
652/652 [==============================] - 242s 371ms/step - loss: 0.2385 - accuracy: 0.9193 - val_loss: 0.5334 - val_accuracy: 0.8195
Epoch 5/5
652/652 [==============================] - 238s 366ms/step - loss: 0.1987 - accuracy: 0.9334 - val_loss: 0.5938 - val_accuracy: 0.8156


## Accuracy and loss

In [76]:
predictions = model.predict(X_val)
print(history.history)

1019/1019 [==============================] - 28s 27ms/step
{'loss': [30452836.0, 0.426649272441864, 0.29697853326797485, 0.2385137528181076, 0.19872897863388062], 'accuracy': [0.6590757369995117, 0.8442339301109314, 0.8970814943313599, 0.9192943572998047, 0.9334228038787842], 'val_loss': [0.5599355697631836, 0.47138699889183044, 0.490759938955307, 0.5333874225616455, 0.5937535166740417], 'val_accuracy': [0.7790697813034058, 0.8236792087554932, 0.8266552090644836, 0.8194759488105774, 0.815640926361084]}


In [77]:
predictions

array([[0.35105407, 0.83397496, 0.250136  ],
       [0.03492784, 0.8690047 , 0.7215559 ],
       [0.00627084, 0.99603564, 0.48747998],
       ...,
       [0.06360538, 0.9689799 , 0.41711906],
       [0.88868815, 0.48623237, 0.13902046],
       [0.23208392, 0.81798524, 0.43959755]], dtype=float32)

In [78]:
m = len(predictions)

In [79]:
m

32594

## Evaluation of the Model on the test data

In [80]:
print("Evaluating on Test data")
results = model.evaluate(X_val, y_val, batch_size=128)
print("Test Loss, Test Accuracy:", results)

Evaluating on Test data
255/255 [==============================] - 7s 28ms/step - loss: 0.5938 - accuracy: 0.8156
Test Loss, Test Accuracy: [0.5937535762786865, 0.815640926361084]


###  So, we can see that we are able to attain the accuracy of 81.56 %.